In [1]:
import sys # Used to overwrite things that are already printed (keep track of how many of something is completed)
import h5py as h # Allows us to read the data from DISCOVR satellite, comes in HDF5 file format
import matplotlib.pyplot as plt # Plotting
import numpy as np # Various math and data analysis uses
from scipy import interpolate  # Allows us to have luminosities anywhere in image,instead of just discrete pixels
import math # only sines and cosines I think, could use numpy for that too
import json # Convert data structures (like lists and dictionaries) into strings and back

In [2]:
# Takes a set of DSCOVR images and for each of them saves data about the brightness of the atmosphere. It writes 3 output 
# files: one with all the brightness data, one with the dates of outliers in the time series (possible lensing events), 
# and one with the time series data for atmosphere and background luminosities. It also shows several relevant plots. Lots
# of details are hidden in the functions nested within this.

def doAll(getInputsFunction):
    
    verbose,allDataOutputFile,outliersOutputFile,timeSeriesOutputFile,usePreviousOutputs,allDataInputFile,\
    outliersInputFile,timeSeriesInputFile,imagesFile,wavelengths,width,gap,plotInnerRing,plotOuterRing,\
    partialTimeSeriesSize,movingMedianBandwidth = getInputsFunction()
    # You can hardcode all these values before starting the run, they are all in one function to make it convenient
            
    files = getFiles(imagesFile) # Get a list of image file names for analysis
    allData = {} # Dictionary to store all desired outputs, will be saved to text file
    outlierImages = [] # Keeps track of images which are outliers in time series, will be saved to text file
    dates = []
    timeSeriesTimes = [] # One time per image, given as Julian Date (JD)
    timeSeriesLums = [] # One lum per image
    timeSeriesBackLums = [] # One background lum per image
    timeSeries = []

    if usePreviousOutputs:
        allData = loadData(allDataInputFile)
        outlierImages = loadData(outliersInputFile)
        writeData(outlierImages,outliersOutputFile) 
        # Write at start so previous outliers are saved even if no new ones arise 
        dates = getAllDates(allDataInputFile)
        timeSeries = loadData(timeSeriesInputFile) # Get previous time series from text file if it exists
        timeSeriesTimes = timeSeries[0]
        timeSeriesLums = timeSeries[1]
        timeSeriesBackLums = timeSeries[2]
        if verbose:
            print("\nTime series from earlier run:")
            plotFromTimeSeries(timeSeriesTimes,timeSeriesLums, 0, 0) # Plot all the time series data that was loaded in
    plottedSoFar = len(timeSeriesLums) # Prevents replotting time series from previous runs again
    previousDates = len(dates)

    if len(files)+len(dates) < 60:
        movingMedianBandwidth = 11 
        # Overrides larger movingMedianBandwidth if that would cut off too much on the sides
        # Not worth looking for outliers if fewer than 11 files

    for file in files:
        date = file.split("_")[2]
        if verbose:
            print("\nDate: ", date)
        time = getTime(date)   # Convert to JD
        lumsByWavelength = [] # List of atmosphere lums across wavelengths for a single image
        backLumsByWavelength = [] # List of background lums across wavelengths for a single image
        try:
            with h.File(file, 'r') as fileForRead: # Automatically closes this file when indent ends
                for wavelength in wavelengths:
                    try:
                        if verbose:
                            print(wavelength)
                        key = date+", "+wavelength

                        xCenter,yCenter,aTrue,bTrue,end = getMetadata(fileForRead,wavelength)
                        # end stands for earth_north_direction, clockwise angle from vertical line to equator
                        # aTrue and bTrue are Earth's semimajor and semiminor axes in the image, respectively
                        if verbose:
                            print("Got metadata.")

                        image = list(fileForRead[wavelength]['Image']) # = getImage(fileForRead, wavelength)
                        # image is a 2048 x 2048 2D array of pixel luminosity values
                        if verbose:
                            print("Got image.")

                        allData[key] = getData(verbose,image,xCenter,yCenter,aTrue,bTrue,end,width,gap,
                                               plotInnerRing,plotOuterRing)
                        # getData returns a dictionary with info for tail profile, tail profiles by sector,
                        # overall atmosphere luminosity, atmosphere lums by sector. This dictionary gets stored
                        # as a value within the larger allData dictionary (it is a nested dictionary) associated
                        # with this "date, wavelength" key.

                        writeData(allData,allDataOutputFile)
                        # Write data every time in case an error causes an interruption before the end.
                        # This is very quick, not too high a time cost to repeat it every time I don't think.
                        if verbose:
                            print("Data written to text file.")

                        lumsByWavelength.append(allData[key]["atmLum"])
                        backLumsByWavelength.append(allData[key]["backLum"])

                        if verbose:
                            plotThisTailData(aTrue,allData,key)
                            # Plot tail profile
                            plotThisSectorTailData(aTrue,allData,key,width,gap)
                            # Plot tail profiles for all sectors, in one plot

                    except KeyError:
                        if not verbose:
                            print("") # Move to next line, don't continue right after "Dates completed: ##"
                        print("No " + wavelength + " for " + date + " or Earth isn't in the image.")

            if not verbose:
                sys.stdout.write("\rDates completed: "+str(len(dates) - previousDates + 1))
                # \r returns to start of line, then you overwrite previous
            dates.append(date)
            timeSeriesTimes.append(time)
            timeSeriesLums.append(np.mean(lumsByWavelength)) # Average of wavelength atm lums used for time series
            timeSeriesBackLums.append(np.mean(backLumsByWavelength))
            timeSeries = [timeSeriesTimes,timeSeriesLums,timeSeriesBackLums]
            writeData(timeSeries,timeSeriesOutputFile) # Write time series to text file
            if verbose:
                print("Time series written to text file.")
            if len(timeSeriesLums) >= movingMedianBandwidth:
                isOutlier = checkForOutlier(verbose,timeSeriesLums,movingMedianBandwidth)
                # Checks if the lum added half a bandwidth ago is an outlier in the time series
                if isOutlier:
                    outlierImages.append(dates[-int(movingMedianBandwidth/2+0.5)])
                    writeData(outlierImages,outliersOutputFile)
                    if verbose:
                        print("Outlier added to outlier dates text file.")
            if verbose:
                if ((len(timeSeriesLums)-plottedSoFar)*len(wavelengths))%partialTimeSeriesSize <= len(wavelengths)/2 or\
                ((len(timeSeriesLums)-plottedSoFar)*len(wavelengths))%partialTimeSeriesSize >=\
                partialTimeSeriesSize-len(wavelengths)/2:
                    # About (partialTimeSeriesSize) images since last TS plot
                    plotFromTimeSeries(timeSeriesTimes,timeSeriesLums, plottedSoFar, 0) 
                    # Plots everything that hasn't been plotted so far
                    print("Partial time series plotted.")
                    plottedSoFar = len(timeSeriesLums)
        except OSError:
            if not verbose:
                print("") # Move to next line, don't continue right after "Dates completed: ##"
            print("Some issue with this file.")

    plotTimeSeriesInDoAll(timeSeriesTimes,timeSeriesLums,partialTimeSeriesSize)
    # Plot time series

    print("Replotting outliers!")
    replotDates(verbose,outliersOutputFile,allDataOutputFile,timeSeriesOutputFile,wavelengths,movingMedianBandwidth,width,
                gap)
    # Replots time series outlier dates, provides extra info so we can check manually if it seems like a lensing event

In [50]:
# Reads text file with data file names, adds data file names to a list, returns list
def getFiles(imagesFile):
    files = []

    with open(imagesFile, "r") as f:
        for x in f:
            files.append(x.strip("\n"))
    return files

In [51]:
# Reads text file with a string that is actually a data structure written out. Loads the string, converts back to data
# structure, returns the data structure. Can be a list or dictionary, probably could be other things too if needed.
def loadData(fileWithDataStructure):
    
    with open(fileWithDataStructure,"r") as dataFileRead:
        stringData = dataFileRead.read()
    data = json.loads(stringData)
    
    return data

In [52]:
# Converts a data structure, like a list or dictionary, into a string. Writes the string to a text file.
def writeData(data,outputFile):
    stringData = json.dumps(data)
    with open(outputFile,"w") as dataFile:
        dataFile.write(stringData)

In [6]:
# Takes the output file from a previous run which contains the allData dictionary, parses it for dates, adds all dates
# to a list, and returns the list.
def getAllDates(allDataFile):
    allData = loadData(allDataFile)
    previousDate = 0
    allDates = []
    for key in allData:
        date = key.split(",")[0]
        if date != previousDate:
            allDates.append(date)
        previousDate = date
    return allDates

In [7]:
# Takes two lists and two positional indices as arguments, plots luminosity vs. time within that index range
# Used for full TS, partial TS, background TS, atmosphere - background TS

def plotFromTimeSeries(timeSeriesTimes,timeSeriesLums, startPosition, endPosition):
    
    sortedTimeSeriesTimes = timeSeriesTimes.copy()
    sortedTimeSeriesTimes.sort()
    
    plt.ylabel('Luminosity')
    plt.xlabel('Time (JD)')
    if endPosition == 0:
        if sortedTimeSeriesTimes == timeSeriesTimes:
            plt.plot(timeSeriesTimes[startPosition:],timeSeriesLums[startPosition:],'r-')
            # Prevents zigzagging red lines across the plot
        size = len(timeSeriesLums[startPosition:])
        if size <= 50 or sortedTimeSeriesTimes != timeSeriesTimes:
            plt.scatter(timeSeriesTimes[startPosition:],timeSeriesLums[startPosition:])
            # If it doesn't make the plot too crowded, or if the red lines would zigzag across the plot
    else:
        if sortedTimeSeriesTimes == timeSeriesTimes:
            plt.plot(timeSeriesTimes[startPosition:endPosition],timeSeriesLums[startPosition:endPosition],'r-')
            # Prevents zigzagging red lines across the plot
        size = len(timeSeriesLums[startPosition:endPosition])
        if size <= 50 or sortedTimeSeriesTimes != timeSeriesTimes:
            plt.scatter(timeSeriesTimes[startPosition:endPosition],timeSeriesLums[startPosition:endPosition])
            # If it doesn't make the plot too crowded, or if the red lines would zigzag across the plot
    plt.title(str(size)+" Time Series Luminosities")
    plt.show()

In [8]:
# Get Julian Date for image
def getTime(date): # Example date: "20200601125518"
    timeRelevantDate = date[:-2] # Last two digits in the "date" are something other than time
    # "202006011255"
    
    year = timeRelevantDate[:4]
    yearContribution = (int(year)-2017)*365 # 2017-2019 are not leap years.
    
    month = timeRelevantDate[4:6]
    if month == "01":
        monthContribution = 0
    if month == "02":
        monthContribution = 31
    if month == "03":
        monthContribution = 60
    if month == "04":
        monthContribution = 91
    if month == "05":
        monthContribution = 121
    if month == "06":
        monthContribution = 152
    if month == "07":
        monthContribution = 182
    if month == "08":
        monthContribution = 213
    if month == "09":
        monthContribution = 244
    if month == "10":
        monthContribution = 274
    if month == "11":
        monthContribution = 305
    if month == "12":
        monthContribution = 335
        
    day = timeRelevantDate[6:8]
    dayContribution = int(day)-1
    
    hour = timeRelevantDate[8:10]
    hourContribution = (int(hour)+12)/24   # +12 because DSCOVR uses GMAT not UTC time conventions
    
    minute = timeRelevantDate[10:]
    minuteContribution = int(minute)/24/60
    
    daysSince2017 = yearContribution + monthContribution + dayContribution + hourContribution + minuteContribution
    
    dateJD = daysSince2017 + 2457754.5   # This is the JD equivalent for 01/01/2017 UTC
    
    return dateJD

In [9]:
# Get xCenter,yCenter,aTrue,bTrue,end directly from DSCOVR data file
def getMetadata(fileForRead,wavelength):
    
    xCenter,yCenter = getAttributesFlipXY(fileForRead,wavelength) # Coords for center of Earth
    
    A = fileForRead[wavelength]['Geolocation']['Earth'] 
    # Data file is structured like a nested dictionary, this gets into the relevant subsection
    
    aTrue = A.attrs['centroid_equatorial_pixel_size']/2 # Metadata has major axis, want semimajor axis 
    # Recall this is really: fileForRead[wavelength]['Geolocation']['Earth'].attrs['centroid_equatorial_pixel_size']/2
    
    bTrue = A.attrs['centroid_polar_pixel_size']/2 # Metadata has minor axis, want semiminor axis
    end = A.attrs['earth_north_direction']
    
    return xCenter,yCenter,aTrue,bTrue,end

In [10]:
# a function to get the xCenter, yCenter using metadata
def getAttributesFlipXY(fileForRead, wavelength):
    # X offset of the center of Earth from the center of the image
    yCenter = float(1024 + fileForRead[wavelength]['Geolocation']['Earth'].attrs['centroid_x_pixel_offset'] )    
    # Y offset of the center of Earth from the center of the image
    xCenter = float(1024 + fileForRead[wavelength]['Geolocation']['Earth'].attrs['centroid_y_pixel_offset'] )

    # Directions are flipped in the metadata, this corrects for that
    
    return xCenter, yCenter

In [11]:
# Gets all brightness data for a given image, stores in dictionary, returns dictionary (which then goes in allData)
def getData(verbose,image,xCenter,yCenter,aTrue,bTrue,end,width,gap,plotInnerRing,plotOuterRing):
    
    tailAs = getTailAs(aTrue,width,gap)
    # list of semimajor axis values for ellipses ranging from slightly smaller to slightly larger than Earth
    
    data = {}
    # Dictionary which will be used to store everything for this image. This whole dictionary will be stored in larger 
    # allData dictionary, which includes all images for the run.
    
    data["tailAs"] = tailAs
    # Store the list of semimajor axis values, it is now the value associated with the key "tailAs" in this dictionary
    
    tailLums, tailLumsBySector, atmLum, atmLumBySector,backLum = getLums(verbose,image,xCenter,yCenter,tailAs,end,aTrue,
                                                                         bTrue,plotInnerRing,plotOuterRing)
    # tailLums: list containing one lum for each full ring specified by the tailAs
    # tailLumsBySector: dictionary with sector names (e.g. "2:30") as keys and sector tailLums lists as values
    # atmLum: a single number representing the atmosphere lum in the image
    # atmLumBySector: dictionary with sector names (e.g. "2:30") as keys and a single atmLum for the sector as a value
    
    # Store all of these in the data dictionary:
    data["tailLums"] = tailLums
    data["tailLumsBySector"] = tailLumsBySector
    data["atmLum"] = atmLum
    data["atmLumBySector"] = atmLumBySector
    data["backLum"] = backLum
    
    return data

In [12]:
# Get a list of semimajor axis values centered around the true semimajor axis from metadata
def getTailAs(aTrue,width,gap):
    tailAs = []
    for n in np.arange(-width, width+gap/100000,gap): # -15 to 15 usually.
        tailAs.append(float(aTrue+n))
    return tailAs

In [13]:
# Gets several of the desired values for each image: tailLums, tailLumsBySector, atmLum, atmLumBySector, backLum
def getLums(verbose,image,xCenter,yCenter,tailAs,end,aTrue,bTrue,plotInnerRing,plotOuterRing):
    
    tailLums = [] # Will store tailLums then be placed in dictionary for this image
    tailLumsBySector = {} # Will store tailLums by sector then be placed in dictionary for this image

    sectorTailLumsLists = [[],[],[],[],[],[],[],[],[],[],[],[]]
    # List of empty lists to start. The inner lists will be tail profiles by sector 
    
    # Adj? Have to adjust a lot for a different number of sectors
# #     numberOfSectors gets inputted, default 12
    
#     sectorTailLumsLists=[]
#     sectors=[]
#     for i in range(numberOfSectors):
#         sectors.append(str(i+1))
#         sectorTailLumsLists.append([]) # List of empty lists to start. The inner lists will be tail profiles by sector 
    
    atmLum = 0 # Will store atmLum then be placed in dictionary for this image
    atmLumBySector = {} # Will store atmLum by sector then be placed in dictionary for this image
    
    aOverB = 1.00336414 # Ratio of equatorial radius to polar radius of Earth
    
    # two dimensional interpolation of the data
    f2D = interpolate.interp2d(range(2048),range(2048),image, kind="linear")
    # Interpolates luminosity data so you can get a luminosity associated with any ordered pair of real 
    # numbers in the image, not just with the 2048 x 2048 discrete pixels
    
    for n in range(len(tailAs)):
        a = tailAs[n]
        b = a/aOverB
        
        # get the luminosities and coordinates for the ring around the center using interpolation
        ringWithoutBinning, ringCoords = getRotatedRing(f2D, xCenter, yCenter, a, b, end)

        # Take the median of every 10 luminosities and only keep track of those medians
        ring = binning(ringWithoutBinning)
        binnedCoords = []
        for i in range(len(ringCoords)):
            if (i%10 == 4):
                binnedCoords.append(ringCoords[i]) # binnedCoords contains the middle point for each of the 360 bins
        
        if n == 0:
            innerBinnedCoords = binnedCoords
        if n == np.median(range(len(tailAs))):
            midBinnedCoords = binnedCoords
        if n == len(tailAs)-1:
            outerBinnedCoords = binnedCoords
            
        k=12 # Would be good to make this adjustable. Number of sectors
        ring2 = nBinning(k,ring)
        sectorTailLumsLists = appendRingToSectorLums(sectorTailLumsLists,ring2)

        tailLums.append(np.mean(ring)) # This is a mean of medians because of binning, still robust to outliers
        
        if verbose:
            sys.stdout.write("\rRing means computed: "+str(n+1))
            # \r returns to start of line, then you overwrite previous
        
    tailLumsBySector = getTailLumsBySectorDictionary(sectorTailLumsLists)
    
    atmAs = getAtmAs(aTrue)
    sectorAtmLumsLists = [[],[],[],[],[],[],[],[],[],[],[],[]]
    
    if verbose:
        print("") # Go to next line, otherwise "atmosphere rings completed" tracker erases "rings completed" tracker
        
    for n in range(len(atmAs)):
        a = atmAs[n] # aTrue, aTrue + 8.5 km, and 4 evenly spaced a's in between
        b = a/aOverB
        
        # get the luminosities and coordinates for the ring around the center using interpolation
        ringWithoutBinning, ringCoords = getRotatedRing(f2D, xCenter, yCenter, a, b, end)

        # Take the median of every 10 luminosities and only keep track of those medians
        ring = binning(ringWithoutBinning)
                    
        k=12 # Would be good to make this adjustable. Number of sectors
        ring2 = nBinning(k,ring)
        sectorAtmLumsLists = appendRingToSectorLums(sectorAtmLumsLists,ring2)
        if verbose:
            sys.stdout.write("\rAtmosphere ring means computed: "+str(n+1))
            # \r returns to start of line, then you overwrite previous

    atmLumBySector = getAtmLumBySectorDictionary(sectorAtmLumsLists) # List of 12 lists of 6 atmLum values each
    # Takes mean of each list, makes it value for associated sector key
    
    atmLums = list(atmLumBySector.values())
    atmLum = np.mean(atmLums)
    
    backLum = getBackLum(image,aTrue,xCenter,yCenter)
    
    if verbose:
        plotRealImage(image,xCenter,yCenter,aTrue,bTrue,end,innerBinnedCoords,midBinnedCoords,outerBinnedCoords,
                      plotInnerRing,plotOuterRing)
        print("The green ring above uses aTrue (and bTrue) in ellipse. Profile goes to +- several pixels from here.\n")
    
    return tailLums, tailLumsBySector, atmLum, atmLumBySector, backLum

In [14]:
# Takes f2D and info about position and orientation of desired ring in image. Returns a list of 3600 luminosities 
# around ring, and the coordinates of the associated points in the image. The ring could be the atmosphere, or 
# slightly smaller or slightly larger.
def getRotatedRing(f2D, xCenter, yCenter, a, b, end):
    phiInDegrees = 90-end # This gives the angle as a CCW angle from the x-axis (end is clockwise offset from y-axis)
    
    phi = phiInDegrees*np.pi/180 # Radians
    
    # iterate from 0 to 360 at intervals of 0.1 
    ring = []
    coords = []
    xCoords = []
    yCoords = []
    for theta in np.arange(0,2*np.pi+0.00001,2*np.pi/3600):
        # using trig to get the x and y coordinates on the ring
        xCoord = xCenter + a * math.cos(phi) * math.cos(theta) - b * math.sin(phi) * math.sin(theta)
        yCoord = yCenter - (a * math.sin(phi) * math.cos(theta) + b * math.cos(phi) * math.sin(theta))
        # Subtract from yCenter to account for y increasing as you go down the image. Make sure you always
        # account for this! (y = 0 is the top, y = 2048 is the bottom.)
        
        # query the interpolated data
        ring.append(f2D(xCoord, yCoord)) # Add luminosities along ring to the list named ring
        coords.append([xCoord, yCoord]) # Add coordinates of points along ring to the list named coords
    
    return ring, coords

In [15]:
# Takes a list, replaces every ten values with one value: their median
def binning(ring):
    newRing = []
    for i in range(10, len(ring)+1, 10):
        newRing.append(np.median(ring[i-10:i])) # Median of first 10 values in ring, then next 10, etc. 
        
    return newRing # List of 360 median luminosities, 3600/10. One median for each 1 degree angle along ring.

In [16]:
# Takes a list, shrinks the length to an inputted number (n). Does this by replacing sections of the list with their mean.
def nBinning(n,ring): # Ring should already have gone through median binning
    newRing = []
    for i in range(round(len(ring)/n), len(ring)+1, round(len(ring)/n)):
        newRing.append(np.mean(ring[i-round(len(ring)/n):i])) # Mean of first few medians in ring, then next few, etc. 
    return newRing # List of n mean medians, representing average luminosity around one nth of the ring

In [17]:
# Works for both tail and atm sectorLumsLists
# Takes a list and distributes its values one by one to the lists within a list of lists the same length
def appendRingToSectorLums(sectorLumsLists,ring2):
    for i in range(len(ring2)):
        lum = ring2[i]
        sectorLumsLists[i].append(lum)
    return sectorLumsLists

In [18]:
# Already have the tail lums by sector as lists within sectorTailLumsLists, this just puts them in a dictionary with their
# keys labelling their sectors
def getTailLumsBySectorDictionary(sectorTailLumsLists):
    tailLumsBySector = {}
    sectors = ["2:30", "1:30", "12:30", "11:30", "10:30", "9:30", "8:30", "7:30", "6:30", "5:30", "4:30", "3:30"]
    # Would be good to make this adjustable
    for i in range(len(sectors)):
        tailLumsBySector[sectors[i]] = sectorTailLumsLists[i]
    return tailLumsBySector

In [19]:
# Gets 6 semi-major axis values, evenly spaced from Earth's surface to 8.5km above Earth's surface (~scale height of 
# atmosphere)
def getAtmAs(aTrue):
    pixelsPerKm = aTrue / 6378.137
    atmHeightInPixels = pixelsPerKm*8.5
    atmAs = []
    for a in np.arange(aTrue,aTrue+atmHeightInPixels+.0001,atmHeightInPixels/5):
        atmAs.append(a)
    return atmAs

In [20]:
# Already have the atm lums by sector as lists within sectorAtmLumsLists, this takes mean and puts it in a 
# dictionary with keys labelling sectors
def getAtmLumBySectorDictionary(sectorAtmLumsLists):
    atmLumBySector = {}
    sectors = ["2:30", "1:30", "12:30", "11:30", "10:30", "9:30", "8:30", "7:30", "6:30", "5:30", "4:30", "3:30"]
    # Would be good to make this adjustable. Number of sectors
    
    for i in range(len(sectors)):
        sector = sectors[i]
        sectorAtmLums = sectorAtmLumsLists[i]
        atmLumBySector[sector] = np.mean(sectorAtmLums)
    
    return atmLumBySector

In [21]:
# Finds background luminosity of an image
def getBackLum(image,aTrue,xCenter,yCenter):
    backPixelLums = []
    for indY, row in enumerate(image):  # Specific y coordinate, and associated row
        if indY%15 == 0: # Reduce the completeness of the background, make this function faster
            for indX, lum in enumerate(row):  # Specific x coordinate in row, and associated pixel luminosity
                if indX%15 == 0: # Reduce the completeness of the background, make this function faster
                    distFromCenter = np.sqrt((indY-yCenter)**2+(indX-xCenter)**2)
    
                    # Find pixels which are in the background, with a buffer of 30 pixels beyond Earth's surface:
                    if distFromCenter > aTrue+30:
                        backPixelLums.append(float(lum))
    backLum = np.median(backPixelLums)
    return backLum

In [22]:
def plotRealImage(image,xCenter,yCenter,aTrue,bTrue,end,innerBinnedCoords,midBinnedCoords,outerBinnedCoords,
                  plotInnerRing,plotOuterRing):
    
    xCoords1,yCoords1,xCoords2,yCoords2 = getLinesYCW(xCenter,yCenter,aTrue,bTrue,end)
    
    ## Show image ##
    
    interpolationType = 'bilinear'
    cmapType = 'gray'

    # create a black and white plot of the data and show it, then save it
    img = plt.imshow(image, interpolation=interpolationType ,cmap=cmapType)

    plt.plot(xCoords1,yCoords1, 'r-')  # Show red equator line
    plt.plot(xCoords2,yCoords2, 'b-')  # Show blue line connecting poles
    
    if plotInnerRing:
        for el in innerBinnedCoords:
            plt.plot(el[0], el[1], 'yo', markersize = 1)
        
    for i in range(len(midBinnedCoords)):
        el = midBinnedCoords[i]
        if i == 0 or i == int(len(midBinnedCoords)/2):
            plt.plot(el[0], el[1], 'ro', markersize = 6)
            if i ==0:
                plt.text(el[0], el[1],'3:00', color="white")
            else: # 1/2
                plt.text(el[0], el[1],'9:00', color="white")
        elif i == int(len(midBinnedCoords)/4) or i == int(3*len(midBinnedCoords)/4):
            plt.plot(el[0], el[1], 'bo', markersize = 6)
            if i ==int(len(midBinnedCoords)/4):
                plt.text(el[0], el[1],'12:00', color="white")
            else: # 3/4
                plt.text(el[0], el[1],'6:00', color="white")
        else:
            plt.plot(el[0], el[1], 'go', markersize = 1)
    
    if plotOuterRing:
        for el in outerBinnedCoords:
            plt.plot(el[0], el[1], 'co', markersize = 1)

    plt.colorbar() # Shows the color scale for luminosities (white is bright, black is dark, and everything in between)
    earthImage = plt.gcf()
    plt.show()
    print("Red is equator, blue is poles.")

In [23]:
def getLinesYCW(xCenter,yCenter,aTrue,bTrue,end):
    if end < 90:
        endPerp = end + 90
    else:
        endPerp = end-90
    
    end = 90-end
    endPerp = 90-endPerp # Adjusts so end represents clockwise angle from vertical line at which equator lies
    
    theta1 = end*np.pi/180   # radians
    theta2 = endPerp*np.pi/180   # radians
    
    xmin1 = xCenter-aTrue*np.cos(theta1)
    xmax1 = xCenter+aTrue*np.cos(theta1)
    
    ymin1 = yCenter+aTrue*np.sin(theta1)
    ymax1 = yCenter-aTrue*np.sin(theta1)  # Switched because y increases as you go down the image
        
    xmin2 = xCenter-bTrue*np.cos(theta2)
    xmax2 = xCenter+bTrue*np.cos(theta2)
    
    ymin2 = yCenter+bTrue*np.sin(theta2)
    ymax2 = yCenter-bTrue*np.sin(theta2)
    
    xCoords1 = [xmin1,xmax1]
    yCoords1 = [ymin1,ymax1]
    xCoords2 = [xmin2,xmax2]
    yCoords2 = [ymin2,ymax2]
    
    return xCoords1,yCoords1,xCoords2,yCoords2

In [24]:
def plotThisTailData(aTrue,allData,key):
    print("Image: " + key)

    plt.title("Tail Profile")
    plt.ylabel('Mean Luminosity')
    plt.xlabel('Equatorial Radius')
    plt.vlines(aTrue, np.min(allData[key]["tailLums"]), np.max(allData[key]["tailLums"]),color = 'g')
    plt.scatter(allData[key]["tailAs"], allData[key]["tailLums"])
    plt.show()

In [25]:
def plotThisSectorTailData(aTrue,allData,key,width,gap):
    print("Image: " + key)

    colors = ['ro','bo','go','yo','co','mo','rx','bx','gx','yx','cx','mx'] # Would be good to make this adjustable
    
    tailLums = []
    
    tailLumsBySector = allData[key]["tailLumsBySector"]
    
    tailAs = getTailAs(aTrue,width,gap)
    
    i=0
    for sector in tailLumsBySector:
        plt.plot(tailAs, tailLumsBySector[sector],colors[i],markersize=3.5,label = sector)
        tailLums += tailLumsBySector[sector]
        i+=1
    
    plt.title("Tail Profiles By Sector")
    plt.ylabel('Mean Luminosity')
    plt.xlabel('Equatorial Radius')
    plt.vlines(aTrue, np.min(tailLums), np.max(tailLums),color = 'k')

    plt.legend()
    plt.show()

In [26]:
# Returns True if the date half a movingMedianBandwidth ago was greater than 4*1.4826 MAD's above the local median 
def checkForOutlier(verbose,timeSeriesLums,movingMedianBandwidth):
    isOutlier = False   # By default, false
    if timeSeriesLums[-int(movingMedianBandwidth/2+0.5)] > np.median(timeSeriesLums[-movingMedianBandwidth:])+\
    4*1.4826*getMAD(timeSeriesLums[-movingMedianBandwidth:]): # 1.4826 is correction factor for using MAD and not StDev
    # Then use 4 sigma cutoff
        isOutlier = True
    if isOutlier:
        if verbose:
            print("The date " + str(int((movingMedianBandwidth-1)/2)) + " before this one is an outlier!") 
        # Break into different types of outlier?
    return isOutlier

In [27]:
# Gets median absolute deviation of a list
def getMAD(x): # x is a list of values
    median = np.median(x)
    deviations = []
    for i in range(len(x)):
        deviations.append(abs(x[i]-median)) # Append all absolute deviations to list "deviations"
    mad = np.median(deviations) # median of all absolute deviations, AKA median absolute deviation, or MAD
    return mad

In [28]:
# Plots full TS and all partial TS from this whole run, including entries loaded in from before 
def plotTimeSeriesInDoAll(timeSeriesTimes,timeSeriesLums,partialTimeSeriesSize):
    remaining = len(timeSeriesLums)
    startPosition = 0
    print("\nPlotting full time series:")
    plotFromTimeSeries(timeSeriesTimes, timeSeriesLums, startPosition, 0) # Plot full time series
    
    print("Plotting partial time series:")
    endPosition = partialTimeSeriesSize
    while remaining>=partialTimeSeriesSize:
        plotFromTimeSeries(timeSeriesTimes, timeSeriesLums, startPosition, endPosition)
        startPosition+=partialTimeSeriesSize
        endPosition+=partialTimeSeriesSize
        remaining-=partialTimeSeriesSize
    if remaining>0:
        plotFromTimeSeries(timeSeriesTimes, timeSeriesLums, startPosition, 0)
    print("\nTime series plots complete.\n\n")

In [29]:
# This is probably the go-to function for reviewing past runs. Give it a file with a subset of dates from the run that 
# you want to review, the allData and time series output files from the run, and make the other parameters match the run.
# This function will replot the profile, profiles by sector, and a local time series for each selected date.

def replotDates(verbose,selectedDatesFile,allDataOutputFile,timeSeriesFile,wavelengths,movingMedianBandwidth,width,gap):
    
    allData = loadData(allDataOutputFile)
    timeSeries = loadData(timeSeriesFile)
    timeSeriesTimes = timeSeries[0]
    timeSeriesLums = timeSeries[1]
    timeSeriesBackLums = timeSeries[2]
    
    try:
        selectedDates = loadData(selectedDatesFile)
        thereAreSelectedDates = True
    except FileNotFoundError:
        thereAreSelectedDates = False
    
    if thereAreSelectedDates:
        allSelectedData = {}
        for key in allData:
            for selectedDate in selectedDates:
                if key.split(",")[0] == selectedDate:
                    allSelectedData[key] = allData[key] # Trimming allData for only selected dates
            if len(allSelectedData) == len(selectedDates)*len(wavelengths):
                if verbose:
                    print("Got allSelectedData.")
                break

        for key in allSelectedData:
            aTrue = np.median(allSelectedData[key]["tailAs"])
            if verbose:
                plotThisTailData(aTrue,allSelectedData,key)
                plotThisSectorTailData(aTrue,allSelectedData,key,width,gap)

            selectedDate = key.split(",")[0]
            
            try:
                recoverLocalTimeSeries(selectedDate,timeSeriesTimes,timeSeriesLums,timeSeriesBackLums,
                                       movingMedianBandwidth)
            except ValueError:
                print(ValueError)
                print("Probably too close to the start or end of the time series to get a full local window.")
                
    else:
        print("No dates selected.")

In [30]:
# Plots local atmosphere lum time series, then local background lum time series, then atm - background TS

def recoverLocalTimeSeries(date,timeSeriesTimes,timeSeriesLums,timeSeriesBackLums,movingMedianBandwidth):
    time = getTime(date)   # Convert to JD
    for i in range(len(timeSeriesTimes)):
        if time == timeSeriesTimes[i]:
            startPosition = i - int((movingMedianBandwidth-1)/2)
            endPosition = i + int((movingMedianBandwidth-1)/2)+1 # Plus one so that the last point is included
            outlierIndex = i
            break
    if startPosition >= 0 and endPosition <= len(timeSeriesTimes): # Make sure it is in plottable range
        # endPosition can equal len(timeSeriesTimes) because the upper bound is not included 
        
        atmLumsMinusBackLums = []
        for i in range(len(timeSeriesLums)):
            atmLumsMinusBackLums.append(timeSeriesLums[i]-timeSeriesBackLums[i])
        
        for lums in [timeSeriesLums,timeSeriesBackLums,atmLumsMinusBackLums]:
        # Atmosphere local time series, then background local TS, then atmosphere - background local TS
            plt.hlines(np.median(lums[startPosition:endPosition]),
                       np.min(timeSeriesTimes[startPosition:endPosition]),
                       np.max(timeSeriesTimes[startPosition:endPosition]),color = 'g',label="Median Lum")
            plt.legend()
            if lums == timeSeriesLums:
                print("\nAtmosphere:\n")
            elif lums == timeSeriesBackLums:
                print("\nBackground:\n")
            else:
                print("\nAtmosphere - Background:\n")
                
            plotFromTimeSeries(timeSeriesTimes,lums,startPosition,endPosition)

            medianLum = np.median(lums[startPosition:endPosition])
            MAD = getMAD(lums[startPosition:endPosition])
            selectedLum = lums[outlierIndex]
            adjustedMAD = 1.4826*MAD
            correctedZScore = (selectedLum-medianLum)/adjustedMAD

            print("Median:",medianLum)
            print("MAD:",MAD)
            print("Selected:",selectedLum)
            print("Corrected Z Score:",correctedZScore,"\n")
        
    else:
        print(date+ " is not in plottable range (too close to start or end of time series).\n")

In [31]:
# Very thorough review of some dates, probably outliers. Gets function center local time series; plots real image, 
# tail profile, and tail profile by sector for all wavelengths using metadata center and function-calculated center.

# This is the first function in this notebook not nested somewhere within doAll, it can be called afterwards to review 
# dates from a doAll run. However, it doesn't just look at the saved data from the run; it also extracts new information
# from the DSCOVR data by running on more wavelengths and using multiple center-finding methods.

def deepInspect(verbose,allImagesFile,dataFolder,datesFile,deepInspectOutputFile):
    # This function assumes that all the data files are in the same folder, dataFolder
    
    wavelengths, width, gap, numberOfSectors,plotInnerRing,plotOuterRing,partialTimeSeriesSize,\
    movingMedianBandwidth = getDeepInspectInputs()

    files = getFiles(allImagesFile) # Get a list of image file names for analysis
    try:
        dates = loadData(datesFile) # List of dates for deep inspection (probably outlier dates)
    except OSError: # Maybe there were no outliers, and the outlier file was never created
        print("No dates file?")
        dates = []
    allDeepInspectData = {} # Dictionary to store all desired outputs, will be saved to text file

    for date in dates:   # the elements in the dates list are the dates of files selected for deep inspection 
        print(date)
        localTimeSeries = getlocalTimeSeriesFxnCenter(verbose,date,files,movingMedianBandwidth)
        timeSeriesTimes,timeSeriesLums,timeSeriesBackLums = localTimeSeries[0],localTimeSeries[1],localTimeSeries[2]
        recoverLocalTimeSeries(date,timeSeriesTimes,timeSeriesLums,timeSeriesBackLums,movingMedianBandwidth)
        file = dataFolder + "epic_1a_" + date + "_03.h5"
        try:    
            with h.File(file, 'r') as fileForRead: # Automatically closes this file when indent ends
                for wavelength in wavelengths:
                    for useFunctionCenter in [False,True]: # Do this image with meta, then function center
                        try:
                            if verbose:
                                print(wavelength)
                            key = date+", "+wavelength

                            xCenter,yCenter,aTrue,bTrue,end = getMetadata(fileForRead,wavelength)
                            # end stands for earth_north_direction, clockwise angle from vertical line to equator
                            # aTrue and bTrue are Earth's semimajor and semiminor axes in the image, respectively
                            if verbose:
                                print("Got metadata.")

                            image = list(fileForRead[wavelength]['Image']) # = getImage(fileForRead, wavelength)
                            # image is a 2048 x 2048 2D array of pixel luminosity values
                            if verbose:
                                print("Got image.")

                            if useFunctionCenter:
                                if verbose:
                                    print("\nUsing Function-Calculated Center.\n")
                                xCenterMeta,yCenterMeta = xCenter,yCenter
                                xCenter,yCenter = calculateCenter(image)
                                key += ", Function Center"
                                plt.plot(xCenterMeta,yCenterMeta,'bo',markersize=3,label="Meta")
                                plt.plot(xCenter,yCenter,'ro',markersize=3,label="Function")
                                plt.legend()
                                plotRealImageWithoutMarkup(image)
                                print("Meta:",(xCenterMeta,yCenterMeta),"Function:",(xCenter,yCenter),"\n")
                            else:
                                if verbose:
                                    print("\nUsing Metadata Center.\n")
                                key += ", Metadata Center"
                                if not verbose:
                                    print(wavelength)

                            allDeepInspectData[key] = getData(verbose,image,xCenter,yCenter,aTrue,bTrue,end,width,
                                                              gap,plotInnerRing,plotOuterRing)
                            # getData returns a dictionary with info for tail profile, tail profiles by sector,
                            # overall atmosphere luminosity, atmosphere lums by sector. This dictionary gets 
                            # stored as a value within the larger allDeepInspectData dictionary 
                            # (it is a nested dictionary) associated with this "date, wavelength" key.

                            writeData(allDeepInspectData,deepInspectOutputFile)
                            # Write data every time in case an error causes an interruption before the end.
                            # This is very quick, not too high a time cost to repeat it every time I don't think.
                            if verbose:
                                print("Data written to text file.")

                                plotThisTailData(aTrue,allDeepInspectData,key)
                                # Plot tail profile

                                plotThisSectorTailData(aTrue,allDeepInspectData,key,width,gap)
                                # Plot tail profiles for all sectors, in one plot

                        except KeyError:
                            print("No " + wavelength + " for " + date + " or Earth isn't in the image.")
        except OSError:
            print("Some issue with this file.")

In [32]:
# Mainly for reviewing outliers, provides very in-depth information

def getDeepInspectInputs():
    
#   No images file here, should use the images file from the regular doAll run
    
    wavelengths = ['Band317nm', 'Band325nm', 'Band340nm', 'Band388nm', 'Band443nm', 'Band551nm', 'Band680nm', 'Band688nm',
                   'Band764nm', 'Band780nm']

    width = 15 # tailAs are +/- this much from aTrue
    gap = 0.5   # separation between consecutive tailAs 
    
    numberOfSectors = 12 # Adjustable functionality for this is not yet ready (as of 08/19/2021)
    
    plotInnerRing = False
    plotOuterRing = False
    
    partialTimeSeriesSize = 100  # Plot time series in the middle of a run, how many new points for each plot?
    
    movingMedianBandwidth = 21 # If len(files) < 60, will automatically be set to 11 regardless of what goes here
    
    return wavelengths, width, gap, numberOfSectors,plotInnerRing,plotOuterRing,partialTimeSeriesSize,\
           movingMedianBandwidth

In [33]:
def getlocalTimeSeriesFxnCenter(verbose,date,files,movingMedianBandwidth):
    for i in range(len(files)):
        if date in files[i]:
            startPosition = i - int((movingMedianBandwidth-1)/2)
            endPosition = i + int((movingMedianBandwidth-1)/2)+1 # Plus one so that the last point is included
            outlierIndex = i
            localFiles = files[startPosition:endPosition]
    
    wavelength = "Band780nm" # This only automatically gets local TS for 780nm wavelength, not yet adjustable either
    
    times = []
    atmLums = []
    backLums = []
    timeSeries = []
    
    print("Getting local time series with function center.")
    if verbose:
        print("Files to complete:",len(localFiles))
    
    mostRecentI = 0 # To print number of files completed at end.
    for i in range(len(localFiles)):
        sys.stdout.write("\rFiles completed: "+str(i)) # \r returns to start of line, then you overwrite previous
        localFile = localFiles[i]
        localDate = localFile.split("_")[2]
        time = getTime(localDate)
        times.append(time)
        with h.File(localFile,'r') as fileForRead:
            
            image = list(fileForRead[wavelength]['Image'])
            f2D = interpolate.interp2d(range(2048),range(2048),image, kind="linear")
            
            xCenter,yCenter,aTrue,bTrue,end = getMetadata(fileForRead,wavelength)
            # end stands for earth_north_direction, clockwise angle from vertical line to equator
            # aTrue and bTrue are Earth's semimajor and semiminor axes in the image, respectively
            
            xCenter,yCenter = calculateCenter(image) # replace metadata center with function center
            
            atmLum = justGetAtmLum(f2D, aTrue, xCenter, yCenter,end)
            atmLums.append(atmLum)
            
            backLum = getBackLum(image,aTrue,xCenter,yCenter)
            backLums.append(backLum)
            mostRecentI = i
    
    sys.stdout.write("\rFiles completed: "+str(mostRecentI+1)) # \r returns to start of line, then you overwrite previous
            
    timeSeries = [times,atmLums,backLums]
    
    return timeSeries

In [34]:
def justGetAtmLum(f2D,aTrue,xCenter,yCenter,end):
    atmAs = getAtmAs(aTrue)
    sectorAtmLumsLists = [[],[],[],[],[],[],[],[],[],[],[],[]]
    aOverB = 1.00336414
    
    for n in range(len(atmAs)):

        a = atmAs[n] # aTrue, aTrue + 8.5 km, and 4 evenly spaced a's in between
        b = a/aOverB

        # get the luminosities and coordinates for the ring around the center using interpolation
        ringWithoutBinning, ringCoords = getRotatedRing(f2D, xCenter, yCenter, a, b, end)

        # Take the median of every 10 luminosities and only keep track of those medians
        ring = binning(ringWithoutBinning)

        k=12 # Would be good to make this adjustable. Number of sectors
        ring2 = nBinning(k,ring)
        sectorAtmLumsLists = appendRingToSectorLums(sectorAtmLumsLists,ring2)

    atmLumBySector = getAtmLumBySectorDictionary(sectorAtmLumsLists) # List of 12 lists of 6 atmLum values each
    # Takes mean of each list, makes it value for associated sector key

    atmLums = list(atmLumBySector.values())
    atmLum = np.mean(atmLums)
    return atmLum

In [35]:
def calculateCenter(image):
    
    threshold = 500
    xCoords = []
    yCoords = []
    
    for indY, row in enumerate(image):
        for indX, el in enumerate(row):
            if el > threshold:
                xCoords.append(indX)
                yCoords.append(indY)
                
    # calculate the center from the image
    xCenter = sum(xCoords)/len(xCoords) # Average x-value for all pixels containing Earth is xCenter
    yCenter = sum(yCoords)/len(yCoords) # Average y-value for all pixels containing Earth is yCenter
    
    return xCenter, yCenter

In [36]:
def plotRealImageWithoutMarkup(image):
    
    interpolationType = 'bilinear'
    cmapType = 'gray'

    # create a black and white plot of the data and show it, then save it
    img = plt.imshow(image, interpolation=interpolationType ,cmap=cmapType)

    plt.colorbar() # Shows the color scale for luminosities (white is bright, black is dark, and everything in between)
    earthImage = plt.gcf()
    plt.show()

In [45]:
# Option to replot everything. If you run with verbose False, this is close to recovering what you 
# would have had with verbose True. It doesn't show the real DSCOVR image though. 

def replotAllDates(verbose,allDatesFile,allDataFile,timeSeriesFile,wavelengths,movingMedianBandwidth,width,gap):
    allDates = getAllDates(allDataFile)
    writeData(allDates,allDatesFile)
    replotDates(verbose,allDatesFile,allDataFile,timeSeriesFile,wavelengths,movingMedianBandwidth,width,gap)

In [38]:
# Can be used to merge allData dicts from different runs, for example. Does not automatically sort by date though!
def mergeDictionariesFromFiles(dictionaryFiles):
    dictionaries = []
    for file in dictionaryFiles:
        dictionary = loadData(file)
        dictionaries.append(dictionary)
    
    mergedDictionarySoFar = dictionaries[0] 
    for dictionary in dictionaries[1:]:
        for key in dictionary:
            mergedDictionarySoFar[key] = dictionary[key]  # The dictionaries should not share any keys
    mergedDictionary = mergedDictionarySoFar
    return mergedDictionary

In [39]:
# Can be used to merge outlier dates lists from different runs, for example. Does not automatically sort by date though!
# Also, this should not be used for time series, because you don't just concatenate those. See next cell for that fxn.
def mergeListsFromFiles(listFiles):
    lists = []
    for file in listFiles:
        List = loadData(file)
        lists.append(List)
    
    mergedListSoFar = lists[0] 
    for List in lists[1:]:
        mergedListSoFar += List  # Just concatenate the lists
    mergedList = mergedListSoFar
    return mergedList

In [57]:
def mergeTimeSeriesFromFiles(timeSeriesFiles):
    mergedTimeSeries1 = mergeListsFromFiles(timeSeriesFiles) # This is [[times],[lums],[backLums],[times],[lums],...]
    timeSeriesTimes = []
    timeSeriesLums = []
    timeSeriesBackLums = []
    for i in range(len(mergedTimeSeries1)):
        if i%3 == 0:
            timeSeriesTimes += mergedTimeSeries1[i]
        elif i%3 == 1:
            timeSeriesLums += mergedTimeSeries1[i]
        else:
            timeSeriesBackLums += mergedTimeSeries1[i]
    mergedTimeSeries = [timeSeriesTimes, timeSeriesLums, timeSeriesBackLums]
    return mergedTimeSeries

In [ ]:
# Adjustable values which may require tweaking between runs

def getDefaultInputs():
    
    verbose = False
    
    # These things need to be edited or checked for each run
    
    imagesFile = "Downloads\Terrascope\data\images.txt"
    
    wavelengths = ['Band780nm']
#   ['Band317nm', 'Band325nm', 'Band340nm', 'Band388nm', 'Band443nm', 'Band551nm', 'Band680nm', 'Band688nm',
#    'Band764nm', 'Band780nm']

    width = 15  # tailAs are +/- this much from aTrue
    gap = 0.5   # separation between consecutive tailAs 
    
#     numberOfSectors = 12 # Adjustable functionality for this is not yet ready (as of 08/19/2021)
    
    plotInnerRing = False
    plotOuterRing = False
    
    partialTimeSeriesSize = 100  # Plot time series in the middle of a run, how many new points for each plot?
    
    movingMedianBandwidth = 21 # If len(files) < 60, will automatically be set to 11 regardless of what goes here
    
    # outputFolderName = "DeletableTest"  # Can use this if not doing a full run
    outputFolderName = "FullRun227to523"    # Make sure you've already created this as an empty folder!

    usePreviousOutputs=False # If false, next lines don't matter (but they need to exist)
    allDataInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\AllData.txt"
    outliersInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\Outliers.txt"
    timeSeriesInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\TimeSeriesOutput.txt"

    # Nothing below usually needs editing

    startOfOutputFilePath = "Downloads\Terrascope\data\OutputFiles" + chr(92) + outputFolderName + chr(92)
    # chr(92) is a forward slash. Can't end a string just by typing that, Python gets confused
    allDataOutputFile = startOfOutputFilePath + "AllData.txt"
    outliersOutputFile = startOfOutputFilePath + "Outliers.txt"
    timeSeriesOutputFile = startOfOutputFilePath + "TimeSeriesOutput.txt"
    
    return verbose,allDataOutputFile,outliersOutputFile,timeSeriesOutputFile,usePreviousOutputs,allDataInputFile,\
           outliersInputFile,timeSeriesInputFile,imagesFile,wavelengths,width,gap,plotInnerRing,plotOuterRing,\
           partialTimeSeriesSize,movingMedianBandwidth

In [42]:
# Adjustable values which may require tweaking between runs

def getInputs():
    
    verbose = False
    
    # These things need to be edited or checked for each run
    
    imagesFile = "Downloads\Terrascope\data\images3.txt"
    
    wavelengths = ['Band780nm']
#   ['Band317nm', 'Band325nm', 'Band340nm', 'Band388nm', 'Band443nm', 'Band551nm', 'Band680nm', 'Band688nm',
#    'Band764nm', 'Band780nm']

    width = 15  # tailAs are +/- this much from aTrue
    gap = 0.5   # separation between consecutive tailAs 
    
#     numberOfSectors = 12 # Adjustable functionality for this is not yet ready (as of 08/19/2021)
    
    plotInnerRing = False
    plotOuterRing = False
    
    partialTimeSeriesSize = 100  # Plot time series in the middle of a run, how many new points for each plot?
    
    movingMedianBandwidth = 21 # If len(files) < 60, will automatically be set to 11 regardless of what goes here
    
#     outputFolderName = "DeletableTest"  # Can use this if not doing a full run
    outputFolderName = "WalkthroughTest"    # Make sure you've already created this as an empty folder!

    usePreviousOutputs=False # If false, next lines don't matter (but they need to exist)
    allDataInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\AllData.txt"
    outliersInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\Outliers.txt"
    timeSeriesInputFile = "Downloads\Terrascope\data\OutputFiles\FullRun523to616\TimeSeriesOutput.txt"

    # Nothing below usually needs editing

    startOfOutputFilePath = "Downloads\Terrascope\data\OutputFiles" + chr(92) + outputFolderName + chr(92)
    # chr(92) is a forward slash. Can't end a string just by typing that, Python gets confused
    allDataOutputFile = startOfOutputFilePath + "AllData.txt"
    outliersOutputFile = startOfOutputFilePath + "Outliers.txt"
    timeSeriesOutputFile = startOfOutputFilePath + "TimeSeriesOutput.txt"
    
    return verbose,allDataOutputFile,outliersOutputFile,timeSeriesOutputFile,usePreviousOutputs,allDataInputFile,\
           outliersInputFile,timeSeriesInputFile,imagesFile,wavelengths,width,gap,plotInnerRing,plotOuterRing,\
           partialTimeSeriesSize,movingMedianBandwidth

In [43]:
def getDeepInspectExpParams(getInputsFunction):
    verbose,allDataOutputFile,outliersOutputFile,timeSeriesOutputFile,usePreviousOutputs,allDataInputFile,\
    outliersInputFile,timeSeriesInputFile,imagesFile,wavelengths,width,gap,plotInnerRing,plotOuterRing,\
    partialTimeSeriesSize,movingMedianBandwidth = getInputsFunction()
    
    startOfOutputFilePath = ""
    splitFileName = allDataOutputFile.split(chr(92))
    for pathPart in splitFileName[:-1]:
        startOfOutputFilePath+= pathPart+chr(92)
    # This gets startOfOutputFilePath without needing the getInputsFunction to return it unnecessarily
    
    return verbose,imagesFile,outliersOutputFile,startOfOutputFilePath

In [ ]:
# This is the main cell for conducting runs
    
print("MAKE SURE ALL PLOTS AND OUTPUTS ARE BEING PRODUCED CORRECTLY BEFORE LETTING IT RUN FOR A LONG TIME!\n")

answer = input("Have you checked inputs? Are you sure you are not overwriting important existing output files? Y or N:\n")


if answer in "Yy":  # Upper or lower case "Y" is acceptable
    answer2 = input("Do you want to run deep inspection of outliers automatically after doAll? Y or N:\n")
    if answer2 in "Yy":
        print("\nDeep inspection of outliers will automatically be conducted at the end!")
    else:
        print("\nNO AUTOMATIC DEEP INSPECTION OF OUTLIERS WILL BE CONDUCTED. IF THIS WAS AN ACCIDENT, START OVER.")
        
    getInputsFunction = getDefaultInputs
#     getInputsFunction = getInputs
    
    doAll(getInputsFunction)
    
    if answer2 in "Yy":
        verbose,allImagesFile,outliersOutputFile,startOfOutputFilePath = getDeepInspectExpParams(getInputsFunction)
        
#         allImagesFile = "Downloads\Terrascope\data\AllImages.txt"  
        # Only needed if this is not the same as what doAll is using. i.e. Only needed when usePreviousInputs is True.
        # If needed, allImagesFile should contain full path for all data files used in previous and current runs.
        # Can also contain other file paths, that won't cause any problems.
        
        dataFolder = "Downloads\Terrascope\data" + chr(92)
        # Folder to look in for DSCOVR data files when doing deep inspection 
        
        deepInspectOutputFile = startOfOutputFilePath + "AllDeepInspectData.txt"
        
        deepInspect(verbose,allImagesFile,dataFolder,outliersOutputFile,deepInspectOutputFile)
        
else:
    print("\nMake changes and try again then!")

MAKE SURE ALL PLOTS AND OUTPUTS ARE BEING PRODUCED CORRECTLY BEFORE LETTING IT RUN FOR A LONG TIME!

